## Scraper

In [1]:
import requests
import pandas as pd

from bs4 import BeautifulSoup as soup

from urllib.parse import urljoin

In [2]:
##hardcode param

#fisherphillips main directory
fp_ppl = 'https://www.fisherphillips.com/en/people/index.html'


#user agent
headers = {
    'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1'
          }
print(headers)

{'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1'}


In [3]:
#hitting the webpage for ppl list
response = requests.get(fp_ppl, headers=headers)
print('Target URL: ',response.url)
print('Status Code: ',response.status_code )
print('\n')
print('Beautifying it... ')

#make it pretty using BS4
souped = soup(response.content, 'html.parser')

print('Done')
print('Closing Requests connection')
#response.close()


Target URL:  https://www.fisherphillips.com/en/people/index.html
Status Code:  200


Beautifying it... 
Done
Closing Requests connection


In [4]:
result = souped.find_all("dt",{"class":"type__h3"})
result[1].a['href']


'https://www.fisherphillips.com/en/people/george-d-adams.html'

In [5]:

print('this isn''t listing all available associates.  Just page 1..')
print('we need a way to traverse through all available pages')

this isnt listing all available associates.  Just page 1..
we need a way to traverse through all available pages


#### Pagination workaround attempt

In [6]:
#manual test for end of page

url = 'https://www.fisherphillips.com/en/people/index.html?f=660'

test = requests.get(url, headers=headers)
print('Target ouf of scope URL: ',test.url)
print('Status Code: ',test.status_code )
print('\n')

#decision
print('this method won''t work as pages are still valid and return status code 200 even if the page has no data ')

Target ouf of scope URL:  https://www.fisherphillips.com/en/people/index.html?f=660
Status Code:  200


this method wont work as pages are still valid and return status code 200 even if the page has no data 


#### Pagination using Mechanize attempt

In [7]:
import mechanize
import re

br = mechanize.Browser()

br.set_header(header='User-Agent',
              value='Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1')

br.set_handle_robots(False)

br.open('https://www.fisherphillips.com/en/people/index.html')

<response_seek_wrapper at 0x2b4a93a66d0 whose wrapped object = <closeable_response at 0x2b4a93a6590 whose fp = <_io.BufferedReader name=2656>>>

In [8]:
#check the links to identify element for Next Page button
i=0
for link in br.links():
    print(i)
    print(link)
    i+=1

0
Link(base_url='https://www.fisherphillips.com/en/people/index.html', url='https://www.fisherphillips.com/', text='', tag='a', attrs=[('href', 'https://www.fisherphillips.com/'), ('aria-label', 'Fisher Phillips LLP')])
1
Link(base_url='https://www.fisherphillips.com/en/people/index.html', url='/people/', text='People', tag='a', attrs=[('href', '/people/'), ('class', 'type__main-nav-link header__navLink--c8e55653')])
2
Link(base_url='https://www.fisherphillips.com/en/people/index.html', url='/services.html', text='Services & Industries', tag='a', attrs=[('href', '/services.html'), ('class', 'type__main-nav-link header__navLink--c8e55653')])
3
Link(base_url='https://www.fisherphillips.com/en/people/index.html', url='/news-insights/', text='Insights', tag='a', attrs=[('href', '/news-insights/'), ('class', 'type__main-nav-link header__navLink--c8e55653')])
4
Link(base_url='https://www.fisherphillips.com/en/people/index.html', url='/innovations-center/', text='Innovation', tag='a', attrs=[

In [9]:
print('not found easily..')
print('we will take the manual route for now')
print('Another area to look into is SELENIUM')

not found easily..
we will take the manual route for now
Another area to look into is SELENIUM


### Selenium attempt

tutorial

Getting started: https://www.youtube.com/watch?v=WnWQgUerR0c&ab_channel=OpenSourceStudent
Page navigation: https://www.youtube.com/watch?v=U6gbGk5WPws&ab_channel=TechWithTim


In [10]:
from selenium import webdriver

from selenium.webdriver.common.by import By

import time

In [101]:
driver = webdriver.Chrome()

In [102]:
#open page
driver.get('https://www.fisherphillips.com/en/people/index.html')

In [14]:
#finding page number element
num_page = driver.find_element(By.LINK_TEXT, '10')
num_page.click()

print('works but we don''t want to always find out what the max page is')

works but we dont want to always find out what the max page is


In [103]:
#finding next page element
next_page = driver.find_element(By.CSS_SELECTOR,'[aria-label="Next page"]')
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
next_page.click()

In [100]:
next_page.click()

In [17]:
driver.current_url

'https://www.fisherphillips.com/en/people/index.html?f=200'

In [18]:
print('this works because we can continously click on ''Next Page'' until the url stops changing')

this works because we can continously click on Next Page until the url stops changing


### Extract profile links out of People page

In [19]:
#list of profiles in pg1 of People
result

[<dt class="type__h3"><a class="styles__anchor--30edf2f2 styles__colorOnHover--9b21850a styles__anchor--30edf2f2" href="https://www.fisherphillips.com/en/people/michael-v-abcarian.html"><span>Michael V. Abcarian</span></a></dt>,
 <dt class="type__h3"><a class="styles__anchor--30edf2f2 styles__colorOnHover--9b21850a styles__anchor--30edf2f2" href="https://www.fisherphillips.com/en/people/george-d-adams.html"><span>George D. Adams</span></a></dt>,
 <dt class="type__h3"><a class="styles__anchor--30edf2f2 styles__colorOnHover--9b21850a styles__anchor--30edf2f2" href="https://www.fisherphillips.com/en/people/vincent-j-adams.html"><span>Vincent J. Adams</span></a></dt>,
 <dt class="type__h3"><a class="styles__anchor--30edf2f2 styles__colorOnHover--9b21850a styles__anchor--30edf2f2" href="https://www.fisherphillips.com/en/people/nazanin-afshar.html"><span>Nazanin Afshar</span></a></dt>,
 <dt class="type__h3"><a class="styles__anchor--30edf2f2 styles__colorOnHover--9b21850a styles__anchor--30e

In [20]:
#profile url extraction 

profiles = []

for item in result:
    
    profiles.append(item.findChild('a')['href'])

print(profiles)
print('\n')
print('successful!')

['https://www.fisherphillips.com/en/people/michael-v-abcarian.html', 'https://www.fisherphillips.com/en/people/george-d-adams.html', 'https://www.fisherphillips.com/en/people/vincent-j-adams.html', 'https://www.fisherphillips.com/en/people/nazanin-afshar.html', 'https://www.fisherphillips.com/en/people/mason-g-alexander.html', 'https://www.fisherphillips.com/en/people/stephanie-r-alexander.html', 'https://www.fisherphillips.com/en/people/erin-gibson-allen.html', 'https://www.fisherphillips.com/en/people/william-e-altman.html', 'https://www.fisherphillips.com/en/people/steven-p-alvarado.html', 'https://www.fisherphillips.com/en/people/christopher-alvarez.html', 'https://www.fisherphillips.com/en/people/emily-alvarez.html', 'https://www.fisherphillips.com/en/people/david-e-amaya.html', 'https://www.fisherphillips.com/en/people/joseph-w-ambash.html', 'https://www.fisherphillips.com/en/people/alyssa-levy.html', 'https://www.fisherphillips.com/en/people/bruce-c-anderson.html', 'https://www.

### Extract Bio and store in dataframe from Profile page

In [21]:
df = pd.DataFrame

In [22]:
type(result)

bs4.element.ResultSet

In [23]:
#hitting the webpage for ppl list
attr_prf = requests.get(profiles[0], headers=headers)
print('Target URL: ',attr_prf.url)
print('Status Code: ',attr_prf.status_code )
print('\n')
print('Beautifying it... ')

#make it pretty using BS4
prf_soup = soup(attr_prf.content, 'html.parser')

print('Done')
print('Closing Requests connection')
#response.close()

Target URL:  https://www.fisherphillips.com/en/people/michael-v-abcarian.html
Status Code:  200


Beautifying it... 
Done
Closing Requests connection


In [24]:
prf_soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="notranslate" name="google"/>
<meta content="Mike Abcarian is managing partner of the firm's Dallas office. For over 30 years he has represented Fortune 500 corporations, units of local government, and local business interests in labor and employment matters." name="description"/>
<meta content="Labor employment workplace wage hour discrimination litigation immigration " name="keywords"/>
<title>Michael V. Abcarian | Fisher Phillips</title>
<link href="https://www.fisherphillips.com/cached/40058/images/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="https://www.fisherphillips.com/cached/40058/images/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="https://www.fisherphillips.com/cached/40058/images/favicon-16x16.png" rel="icon" size

In [25]:
#bio - has all profile data
bio = prf_soup.find_all("div", {"class": "rte"})
bio

[<div class="rte">
 <p>Mike Abcarian is managing partner of the firm's Dallas office. For over 30 years he has represented Fortune 500 corporations, units of local government, and local business interests in labor and employment matters.</p>
 <p>He has handled hundreds of lawsuits in federal and state courts with an exceptional success record, including lead counsel defense of complex litigation and nationwide class actions.</p>
 <p>Many of Mike's successful cases resulted in defense verdicts for employer clients following trial by jury. Mike also handles complex workplace safety matters, including fatality investigations, and has represented employers in high-visibility proceedings before the Occupational Safety and Health Administration (OSHA).</p>
 <p>He has handled significant compensation compliance matters--some involving thousands of employees--in proceedings before the Wage &amp; Hour Division of the U. S. Department of Labor (USDOL). Mike also appears frequently before the Equ

In [26]:
#get overview

#clean up tags
overview = [str(x.text) for x in bio[0].find_all('p')]
overview = ' '.join(overview).replace(u'\xa0', u' ')

#unicode/charactor encoding  : look into unicodedata.normalize
overview 

'Mike Abcarian is managing partner of the firm\'s Dallas office. For over 30 years he has represented Fortune 500 corporations, units of local government, and local business interests in labor and employment matters. He has handled hundreds of lawsuits in federal and state courts with an exceptional success record, including lead counsel defense of complex litigation and nationwide class actions. Many of Mike\'s successful cases resulted in defense verdicts for employer clients following trial by jury. Mike also handles complex workplace safety matters, including fatality investigations, and has represented employers in high-visibility proceedings before the Occupational Safety and Health Administration (OSHA). He has handled significant compensation compliance matters--some involving thousands of employees--in proceedings before the Wage & Hour Division of the U. S. Department of Labor (USDOL). Mike also appears frequently before the Equal Employment Opportunity Commission (EEOC) defe

In [27]:
#get education

tmp = []

#for loop for possible iteration
target = bio[1].find('h3',string='Education')
for sib in target.find_next_siblings():
    if sib.name=="h3":
        break
    else:
        
        tmp.append(sib.text)
        
for i in tmp:
    x = i.split("\n")

edu = [z for z in x if z != '']
edu

['J.D., 1977, Boston College Law School',
 'A.B., 1974, with honors, University of California, Berkeley']

In [28]:
#get bar admissions

tmp = []

#for loop for possible iteration
target = bio[1].find('h3',string='Bar Admissions')
for sib in target.find_next_siblings():
    if sib.name=="h3":
        break
    else:
        
        tmp.append(sib.text)
        
for i in tmp:
    x = i.split("\n")

bar = [z for z in x if z != '']
bar

['Georgia', 'Texas']

In [46]:
#get court admissions

tmp = []

#for loop for possible iteration
target = bio[1].find('h3',string='Court Admissions')
for sib in target.find_next_siblings():
    if sib.name=="h3":
        break
    else:
        tmp.append(sib.text)
        
for i in tmp:
    x = i.split("\n")

court = [z for z in x if z != '']
court

['Florida Supreme Court',
 'Georgia Court of Appeals',
 'Georgia Supreme Court',
 'Texas Supreme Court',
 'Texas Courts of Appeals',
 'U.S. Court of Appeals for the Eighth Circuit',
 'U.S. Court of Appeals for the Fifth Circuit',
 'U.S. Court of Appeals for the Seventh Circuit',
 'U.S. Court of Appeals for the Tenth Circuit',
 'U.S. District Court for the Eastern District of Texas',
 'U.S. District Court for the Middle District of Florida',
 'U.S. District Court for the Middle District of Georgia',
 'U.S. District Court for the Northern District of Georgia',
 'U.S. District Court for the Northern District of Texas',
 'U.S. District Court for the Southern District of Texas',
 'U.S. District Court for the Western District of Texas',
 'U.S. Supreme Court']

In [30]:
#get recognitions

recog = []

for item in bio[2].find_all('li'):
    recog.append(item.text)
    
recog

['Recognized in The Best Lawyers in America (2013 – 2024)',
 'Inducted as a Fellow into The College of Labor and Employment Lawyers (2018)',
 'Chambers USA (2016-present)',
 'Texas Super Lawyers (2005-present)']

In [31]:
#get affiliations

affil = []

for item in bio[3].find_all('li'):
    affil.append(item.text)
    
affil

['Board Certified in Labor and Employment Law by the Texas Board of Legal Specialization',
 'Member, American Bar Association, Labor and Employment Law Section',
 'Member, State Bar of Texas, Labor and Employment Law Section',
 'Fellow of the American Bar Foundation']

In [33]:
#name
#<meta content="Michael V. Abcarian" property="og:title"/>

name = prf_soup.find("meta", {"property": "og:title"})['content']

In [34]:
print(prf_soup.find("meta", {"property": "og:title"})['content'])

Michael V. Abcarian


In [38]:
prf_soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="notranslate" name="google"/>
<meta content="Mike Abcarian is managing partner of the firm's Dallas office. For over 30 years he has represented Fortune 500 corporations, units of local government, and local business interests in labor and employment matters." name="description"/>
<meta content="Labor employment workplace wage hour discrimination litigation immigration " name="keywords"/>
<title>Michael V. Abcarian | Fisher Phillips</title>
<link href="https://www.fisherphillips.com/cached/40058/images/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="https://www.fisherphillips.com/cached/40058/images/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="https://www.fisherphillips.com/cached/40058/images/favicon-16x16.png" rel="icon" size

In [42]:
#focus - has all focus data
focus = prf_soup.find("h2", string= "Service Focus")
focus

<h2 class="type__h5">Service Focus</h2>

In [43]:
#get service focus

tmp = []

#for loop for possible iteration
focus = prf_soup.find("h2", string= "Service Focus")
for sib in focus.find_next_siblings():
    if sib.name=="h2":
        break
    else:
        tmp.append(sib.text)
        
for i in tmp:
    x = i.split("\n")

serFoc = [z for z in x if z != '']
serFoc

['Class and Collective Actions',
 'Labor Relations',
 'Litigation and Trials',
 'Workplace Safety and Catastrophe Management',
 'Wage and Hour']

In [44]:
#get industry focus

tmp = []

#for loop for possible iteration
focus = prf_soup.find("h2", string= "Industry Focus")
for sib in focus.find_next_siblings():
    if sib.name=="h2":
        break
    else:
        tmp.append(sib.text)
        
for i in tmp:
    x = i.split("\n")

indFoc = [z for z in x if z != '']
indFoc

['Education']

### Building DataFrame

In [59]:
fpbio = pd.DataFrame(columns=['name','overview','serviceFocus','industryFocus','education','barAdmissions','courtAdmissions','recognitions','affiliations'])
fpbio

,name,overview,serviceFocus,industryFocus,education,barAdmissions,courtAdmissions,recognitions,affiliations


In [60]:
new = {
    'name':name,
    'overview':overview,
    'serviceFocus':serFoc,
    'industryFocus':indFoc,
    'education':edu,
    'barAdmissions':bar,
    'courtAdmissions':court,
    'recognitions':recog,
    'affiliations':affil
}

df = pd.DataFrame.from_dict(new, orient='index')
df = df.transpose()
df

,name,overview,serviceFocus,industryFocus,education,barAdmissions,courtAdmissions,recognitions,affiliations
0,Michael V. Abcarian,Mike Abcarian is managing partner of the firm'...,"[Class and Collective Actions, Labor Relations...",[Education],"[J.D., 1977, Boston College Law School, A.B., ...","[Florida Supreme Court, Georgia Court of Appea...","[Florida Supreme Court, Georgia Court of Appea...",[Recognized in The Best Lawyers in America (20...,[Board Certified in Labor and Employment Law b...


In [61]:
fpbio = pd.concat([fpbio,df], ignore_index=False)
fpbio.reset_index()

,index,name,overview,serviceFocus,industryFocus,education,barAdmissions,courtAdmissions,recognitions,affiliations
0,0,Michael V. Abcarian,Mike Abcarian is managing partner of the firm'...,"[Class and Collective Actions, Labor Relations...",[Education],"[J.D., 1977, Boston College Law School, A.B., ...","[Florida Supreme Court, Georgia Court of Appea...","[Florida Supreme Court, Georgia Court of Appea...",[Recognized in The Best Lawyers in America (20...,[Board Certified in Labor and Employment Law b...


## Putting it altogether

In [29]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

bs4.element.ResultSet

In [140]:
#fisherphillips main directory
fp_ppl = 'https://www.fisherphillips.com/en/people/index.html'


#user agent
headers = {
    'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1'
          }

#create container dataframe
fpbio = pd.DataFrame(columns=['name','overview','serviceFocus','industryFocus','education','barAdmissions','courtAdmissions','recognitions','affiliations'])
fpbio

,name,overview,serviceFocus,industryFocus,education,barAdmissions,courtAdmissions,recognitions,affiliations


In [65]:

#hitting the webpage for directory list
response = requests.get(fp_ppl, headers=headers)
print('Target URL: ',response.url)
print('Status Code: ',response.status_code )
print('\n')

print('Beautifying it... ')
#make it pretty using BS4
souped = soup(response.content, 'html.parser')
print('Done')
print('\n')

print('Closing Requests connection')
response.close()
print('\n')

#list of people bio tag and link in pg1 of directory
result = souped.find_all("dt",{"class":"type__h3"})
#print(result)

#people url extraction 
people = []
for item in result: people.append(item.findChild('a')['href'])
print('Extracting urls out of html tag...')
#print(people)
print('successful!')
print('\n')



Target URL:  https://www.fisherphillips.com/en/people/index.html
Status Code:  200


Beautifying it... 
Done


Closing Requests connection


Extracting urls out of html tag...
successful!




In [106]:
#Selenium for subsequent page url extraction

#set base case
current_page = fp_ppl
page_links = []
page_links.append(current_page)

#open browser
driver = webdriver.Chrome()

#open page
driver.get(fp_ppl)
time.sleep(5)

#finding next page element
next_button = driver.find_element(By.CSS_SELECTOR,'[aria-label="Next page"]')
#scroll to bottom
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
next_button.click()

next_page = driver.current_url

#make it iterative

#while next_page != current_page:
#getting 504 Gateway Time Out.  Need to look into this

for i in range(5):
    driver.get(next_page)
    time.sleep(5)

    current_page = next_page
    page_links.append(current_page)

    next_button = driver.find_element(By.CSS_SELECTOR,'[aria-label="Next page"]')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    next_button.click()
    next_page = driver.current_url
   

In [109]:
 page_links

['https://www.fisherphillips.com/en/people/index.html',
 'https://www.fisherphillips.com/en/people/index.html?f=20',
 'https://www.fisherphillips.com/en/people/index.html?f=40',
 'https://www.fisherphillips.com/en/people/index.html?f=60',
 'https://www.fisherphillips.com/en/people/index.html?f=80',
 'https://www.fisherphillips.com/en/people/index.html?f=100']

In [144]:

for url in page_links:
    
    response = requests.get(url, headers=headers)
    souped = soup(response.content, 'html.parser')
    response.close()

    result = souped.find_all("dt",{"class":"type__h3"})

    people = []
    for item in result: people.append(item.findChild('a')['href'])

    for p in people:

        #hitting the webpage for ppl list
        person = requests.get(p, headers=headers)

        per_soup = soup(person.content, 'html.parser')
        person.close()

        #bio - has all profile data
        bio = per_soup.find_all("div", {"class": "rte"})

        #print(bio)
        print(p)

        #get overview
        
        try:
            overview = [str(x.text) for x in bio[0].find_all('p')]
            overview = ' '.join(overview).replace(u'\xa0', u' ')
        except IndexError:
            overview = 'null'

        #get education
        tmp = []
        #for loop for possible iteration
        try:
            target = bio[1].find('h3',string='Education')
            if target is not None:
                for sib in target.find_next_siblings():
                    if sib.name=="h3":
                        break
                    else:
                        tmp.append(sib.text)

                for i in tmp:
                    x = i.split("\n")

                edu = [z for z in x if z != '']
        except IndexError:
            edu = []
                
                
        #get bar admissions
        tmp = []
        #for loop for possible iteration
        try:
            target = bio[1].find('h3',string='Bar Admissions')
            if target is not None:
                for sib in target.find_next_siblings():
                    if sib.name=="h3":
                        break
                    else:
                        tmp.append(sib.text)

                for i in tmp:
                    x = i.split("\n")

                bar = [z for z in x if z != '']
        except IndexError:
            bar = []

        #get court admissions
        tmp = []
        #for loop for possible iteration
        try:
            target = bio[1].find('h3',string='Court Admissions')
            if target is not None:
                for sib in target.find_next_siblings():
                    if sib.name=="h3":
                        break
                    else:
                        tmp.append(sib.text)

                for i in tmp:
                    x = i.split("\n")

                court = [z for z in x if z != '']
        except IndexError:
            court = []
            
        #get recognitions
        recog = []  
        try:
            for item in bio[2].find_all('li'):
                recog.append(item.text)
        except IndexError:
            recog.append('null')

        #get affiliations
        affil = []
        try:
            for item in bio[3].find_all('li'):
                affil.append(item.text)
        except IndexError:
            affil.append('null')

        #get name
        name = per_soup.find("meta", {"property": "og:title"})['content']

        #get service focus
        tmp = []
        #for loop for possible iteration
        focus = per_soup.find("h2", string= "Service Focus")
        if focus is not None:
            for sib in focus.find_next_siblings():
                if sib.name=="h2":
                    break
                else:
                    tmp.append(sib.text)

            for i in tmp:
                x = i.split("\n")

            serFoc = [z for z in x if z != '']

        #get industry focus
        tmp = []

        #for loop for possible iteration
        focus = per_soup.find("h2", string= "Industry Focus")
        if focus is not None:
            for sib in focus.find_next_siblings():
                if sib.name=="h2":
                    break
                else:
                    tmp.append(sib.text)

            for i in tmp:
                x = i.split("\n")

            indFoc = [z for z in x if z != '']


        #create dataframe and concat
        new = {
        'name':name,
        'overview':overview,
        'serviceFocus':serFoc,
        'industryFocus':indFoc,
        'education':edu,
        'barAdmissions':bar,
        'courtAdmissions':court,
        'recognitions':recog,
        'affiliations':affil
        }

        df = pd.DataFrame.from_dict(new, orient='index')
        df = df.transpose()

        fpbio = pd.concat([fpbio,df], ignore_index=True)


https://www.fisherphillips.com/en/people/michael-v-abcarian.html
https://www.fisherphillips.com/en/people/george-d-adams.html
https://www.fisherphillips.com/en/people/vincent-j-adams.html
https://www.fisherphillips.com/en/people/nazanin-afshar.html
https://www.fisherphillips.com/en/people/mason-g-alexander.html
https://www.fisherphillips.com/en/people/stephanie-r-alexander.html
https://www.fisherphillips.com/en/people/erin-gibson-allen.html
https://www.fisherphillips.com/en/people/william-e-altman.html
https://www.fisherphillips.com/en/people/steven-p-alvarado.html
https://www.fisherphillips.com/en/people/christopher-alvarez.html
https://www.fisherphillips.com/en/people/emily-alvarez.html
https://www.fisherphillips.com/en/people/david-e-amaya.html
https://www.fisherphillips.com/en/people/joseph-w-ambash.html
https://www.fisherphillips.com/en/people/alyssa-levy.html
https://www.fisherphillips.com/en/people/bruce-c-anderson.html
https://www.fisherphillips.com/en/people/lucy-anderson.html

SSLError: HTTPSConnectionPool(host='www.fisherphillips.com', port=443): Max retries exceeded with url: /en/people/marianna-bertikian.html (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2576)')))

In [ ]:
print('success')

In [113]:
people

['https://www.fisherphillips.com/en/people/julia-w-clark.html',
 'https://www.fisherphillips.com/en/people/megan-f-clark.html',
 'https://www.fisherphillips.com/en/people/jerry-paul-cline.html',
 'https://www.fisherphillips.com/en/people/sheyron-b-cobb.html',
 'https://www.fisherphillips.com/en/people/brandy-cody.html',
 'https://www.fisherphillips.com/en/people/j-randall-coffey.html',
 'https://www.fisherphillips.com/en/people/tillman-y-coffey.html',
 'https://www.fisherphillips.com/en/people/kendall-cole.html',
 'https://www.fisherphillips.com/en/people/patrick-j-collopy.html',
 'https://www.fisherphillips.com/en/people/theresa-m-connolly.html',
 'https://www.fisherphillips.com/en/people/christopher-conti.html',
 'https://www.fisherphillips.com/en/people/collin-d-cook.html',
 'https://www.fisherphillips.com/en/people/jessica-t-cook.html',
 'https://www.fisherphillips.com/en/people/laurel-k-cornell.html',
 'https://www.fisherphillips.com/en/people/brent-a-cossrow.html',
 'https://www.

In [145]:
fpbio

,name,overview,serviceFocus,industryFocus,education,barAdmissions,courtAdmissions,recognitions,affiliations
0,Michael V. Abcarian,Mike Abcarian is managing partner of the firm'...,"[Class and Collective Actions, Labor Relations...",[Education],"[J.D., 1977, Boston College Law School, A.B., ...","[Georgia, Texas]","[Florida Supreme Court, Georgia Court of Appea...",[Recognized in The Best Lawyers in America (20...,[Board Certified in Labor and Employment Law b...
1,George D. Adams,George Adams is one of the four founding partn...,"[Labor Relations, Litigation and Trials, Couns...",[Manufacturing],"[J.D., 2000, cum laude, University of Louisvil...",[Kentucky],"[Kentucky Supreme Court, U.S. Court of Appeals...",[Recognized in The Best Lawyers in America (20...,"[Kentucky Association of Manufacturers, Board ..."
2,Vincent J. Adams,Vincent Adams is an Of Counsel in the firm's S...,"[Labor Relations, Litigation and Trials, Couns...",[Manufacturing],"[J.D., 2006, Pepperdine University Caruso Scho...",[California],[U.S. District Court for the Central District ...,[Selected for inclusion in Rising Stars for No...,[Contra Costa Bar Association ]
3,Nazanin Afshar,Nazanin Afshar is an associate in the firm’s L...,"[Class and Collective Actions, Employment Disc...",[Manufacturing],"[J.D., 2012, University of California, Hasting...",[California],"[California Courts of Appeal, California Super...","[Recognized in The Best Lawyers in America, On...","[Member, American Bar Association]"
4,Mason G. Alexander,Mason G. Alexander is a partner in the Charlot...,"[Employment Discrimination and Harassment, Lab...","[Automotive Dealership, Manufacturing]","[J.D., 1985, University of South Carolina Scho...","[North Carolina, South Carolina]","[U.S. Court of Appeals for the Fourth Circuit,...","[Chambers USA, Labor & Employment (2008 – 2023...",[President of the Gaston County Personnel Asso...
...,...,...,...,...,...,...,...,...,...
227,Phillip C. Bauknight,Phil represents and serves as trusted counsel ...,"[Cryptocurrency and Blockchain, Employment Dis...",[Manufacturing],"[J.D., 2008, Rutgers Law School, B.A., 2005, R...","[New Jersey, New York]","[U.S. Court of Appeals for the Second Circuit,...",[Honored as a 2023 “Leader in Technology” by N...,"[National Black Lawyers Top 40 Under 40, Execu..."
228,Cheryl L. Behymer,Cheryl Behymer is a Partner in the Columbia of...,[Affirmative Action and Federal Contract Compl...,"[Education, Healthcare, Manufacturing]","[J.D., 1994, cum laude, Catholic University of...","[Maryland, South Carolina]","[U.S. Supreme Court, Maryland Court of Appeals...",[Recognized in The Best Lawyers in America (20...,[Certified as a Labor and Employment Specialis...
229,Clarence M. Belnavis,Clarence Belnavis is the regional managing par...,"[Counseling and Advice, Employment Discriminat...","[Automotive Dealership, Financial Services, He...","[J.D., 1994, Howard University School of Law, ...","[Oregon, Washington]",[U.S. Court of Appeals for the Federal Circuit...,[Recognized in The Best Lawyers in America (20...,"[President, Oregon Chapter of the National Bar..."
230,Sarah Bennett,Sarah is an associate in the Fisher Phillips’ ...,"[Employment Discrimination and Harassment, Lit...","[Supply Chain, Transportation and Supply Chain]","[J.D., 2019, Chapman University School of Law,...","[California, Michigan]",[U.S. Court of Appeals for the Federal Circuit...,[null],[null]


In [148]:
fpbio.to_csv('out1.csv', mode='w+')